# Laboratory of Computational Physics mod. B
# Title????
Authors:
Valeria Fioroni, 
Matteo Guida,
Philipp Zehetner

## Scattered notes
We have the the total number of events are:
$$N = \int L \, dt \cdot \sigma$$


In [1]:
import pandas as pd
import os
from root_pandas import read_root
#import tensorflow as tf
import seaborn as sns
import numpy as np

Welcome to JupyROOT 6.16/00


# Read Data

In [2]:
#Read background and signal samples from ROOT files
#BACKGROUND
'''CONTINUUM BACKGROUND'''
Continuum_bkg = pd.DataFrame() #empty dataframe to contain continuum bkg samples
#export all continuum bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Background/Continuum/"):
    continuum_temp = pd.DataFrame()
    continuum_temp = read_root("../DataBelle2/Background/Continuum/" + file, 'B0')
    frames=[Continuum_bkg, continuum_temp]
    Continuum_bkg= pd.concat(frames, ignore_index=True)
    
'''PEAKING BACKGROUND'''
Peaking_bkg = pd.DataFrame() #empty dataframe to contain peaked bkg samples
#export all peaking bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Background/Peaking/"):
    peaking_temp = pd.DataFrame()
    peaking_temp = read_root("../DataBelle2/Background/Peaking/" + file, 'B0')
    frames=[Peaking_bkg, peaking_temp]
    Peaking_bkg= pd.concat(frames, ignore_index=True)    


'''SIGNAL'''
Signal = pd.DataFrame() #empty dataframe to contain signal samples
#export all peaking bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Signal/"):
    signal_temp = pd.DataFrame()
    signal_temp = read_root("../DataBelle2/Signal/" + file, 'B0')
    frames=[Signal, signal_temp]
    Signal= pd.concat(frames, ignore_index=True) 


#Signal: require B0_isSignal == True
Signal=Signal[Signal['B0_isSignal']== True]
#reset row indexing
Signal.reset_index(drop=True, inplace=True)
print(Signal.shape)

#Continuum: require B0_isContinuumEvent == True
Continuum_bkg=Continuum_bkg[Continuum_bkg['B0_isContinuumEvent']== True]
#reset row indexing
Continuum_bkg.reset_index(drop=True, inplace=True)
print(Continuum_bkg.shape)

#Peaking: require B0_isNotContinuumEvent == True
Peaking_bkg=Peaking_bkg[Peaking_bkg['B0_isNotContinuumEvent']== True]
#reset row indexing
Peaking_bkg.reset_index(drop=True, inplace=True)
print(Peaking_bkg.shape)


# '''DNN Error: Input contains NaN, infinity or a value too large for dtype('float64')'''
# #Check if there are NaN in datasets (see below why)
# Peaking_bkg.replace([np.inf, -np.inf], np.nan)
# Continuum_bkg.replace([np.inf, -np.inf], np.nan)
# Signal.replace([np.inf, -np.inf], np.nan)
# N_peak=Peaking_bkg.isnull().T.any().T.sum()
# print(N_peak)
# N_cont=Continuum_bkg.isnull().T.any().T.sum()
# print(N_cont)
# N_sig=Signal.isnull().T.any().T.sum()
# print(N_sig)


Important_variables=[
"B0_Pcms","B0_P4cms",
"B0_etap_Pcms" ,
"B0_etap_P4cms",
"B0_etap_eta_gamma0_Pcms",
"B0_etap_eta_gamma0_P4cms",   
"B0_etap_eta_gamma1_Pcms",  
"B0_etap_eta_gamma1_P4cms",            
"B0_etap_pi1_Pcms",                     
"B0_K_S0_P4cms",
"B0_K_S0_M",                
"B0_M",          
"B0_etap_M",     
"B0_etap_eta_M",
"B0_TrCSMVA"]

/home/utente/.local/lib/python3.6/site-packages/root_pandas/readwrite.py:320: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(columns, axis=1, copy=False)


(20491, 391)
(1847, 391)
(208, 391)


In [3]:
#Select variables useful for the Classification task
Signal_selection = Signal[Important_variables]
Continuum_bkg_selection= Continuum_bkg[Important_variables]
Peaking_bkg_selection= Peaking_bkg[Important_variables]

print(Continuum_bkg_selection.shape)
print(Peaking_bkg_selection.shape)
print(Signal_selection.shape)


#The Dataframe contains arrays as elements, I create a new column for each component
Signal_comp = pd.concat([pd.DataFrame(Signal_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Signal_selection[c] for c in Signal_selection], axis=1)
Peaking_bkg_comp = pd.concat([pd.DataFrame(Peaking_bkg_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Peaking_bkg_selection[c] for c in Peaking_bkg_selection], axis=1)
Continuum_bkg_comp = pd.concat([pd.DataFrame(Continuum_bkg_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Continuum_bkg_selection[c] for c in Continuum_bkg_selection], axis=1)

(1847, 15)
(208, 15)
(20491, 15)


In [4]:
from IPython.display import display
pd.options.display.max_columns = None
display(Signal_comp)

,B0_Pcms,B0_P0cms,B0_P1cms,B0_P2cms,B0_P3cms,B0_etap_Pcms,B0_etap_P0cms,B0_etap_P1cms,B0_etap_P2cms,B0_etap_P3cms,B0_etap_eta_gamma0_Pcms,B0_etap_eta_gamma0_P0cms,B0_etap_eta_gamma0_P1cms,B0_etap_eta_gamma0_P2cms,B0_etap_eta_gamma0_P3cms,B0_etap_eta_gamma1_Pcms,B0_etap_eta_gamma1_P0cms,B0_etap_eta_gamma1_P1cms,B0_etap_eta_gamma1_P2cms,B0_etap_eta_gamma1_P3cms,B0_etap_pi1_Pcms,B0_K_S0_P0cms,B0_K_S0_P1cms,B0_K_S0_P2cms,B0_K_S0_P3cms,B0_K_S0_M,B0_M,B0_etap_M,B0_etap_eta_M,B0_TrCSMVA
0,0.270603,0.235256,-0.117511,-0.063811,5.306784,2.472725,0.017698,2.448359,0.345819,2.651747,0.992038,0.169390,0.973137,0.091929,0.992038,0.321009,-0.225019,0.206316,0.099232,0.321009,0.315213,0.217558,-2.565870,-0.409629,2.655038,0.500407,5.299881,0.956255,0.541614,2.585103
1,0.246755,0.032232,-0.243893,-0.019109,5.294516,2.425779,-0.798072,2.076690,-0.966872,2.607609,1.384245,-0.359192,1.247643,-0.480108,1.384245,0.077783,-0.050287,-0.047548,0.035504,0.077783,0.292910,0.830303,-2.320584,0.947763,2.686908,0.496696,5.288764,0.959869,0.576153,2.202004
2,0.234618,0.180495,-0.149116,0.015220,5.308913,2.619631,0.753423,-1.646525,1.893087,2.789253,0.749589,0.024481,-0.583025,0.470495,0.749589,0.599604,0.307091,-0.082348,0.508368,0.599604,0.745510,-0.572928,1.497409,-1.877867,2.519659,0.501817,5.303726,0.962386,0.556313,3.055712
3,0.337770,-0.008184,0.325288,-0.090606,5.311382,2.684981,-0.252916,1.795572,-1.980171,2.850605,1.769062,-0.281540,0.977534,-1.447322,1.769062,0.179530,0.085756,0.156609,-0.018723,0.179530,0.380820,0.244732,-1.470283,1.889565,2.460777,0.513165,5.300632,0.956870,0.568429,0.500245
4,0.271372,0.112881,0.073488,0.235584,5.297682,2.483211,-2.309625,-0.826704,0.385399,2.661494,0.413559,-0.401977,0.047468,-0.084808,0.413559,1.113338,-0.900850,-0.532913,0.379466,1.113338,0.779205,2.422506,0.900192,-0.149815,2.636188,0.498159,5.290727,0.957299,0.558190,5.201922
5,0.388067,-0.331670,-0.107813,-0.170197,5.264018,2.504867,-0.316183,-1.784415,1.729234,2.682027,1.339800,-0.320374,-0.904200,0.935332,1.339800,0.204704,0.099083,0.004066,0.179080,0.204704,0.480913,-0.015487,1.676602,-1.899431,2.581991,0.497600,5.249694,0.958983,0.533261,0.665817
6,0.333627,0.054735,-0.328583,-0.018547,5.289863,2.476586,-1.506793,0.322077,1.938896,2.655307,1.310162,-0.715114,0.512443,0.970845,1.310162,0.582764,-0.360026,-0.125153,0.440832,0.582764,0.157781,1.561528,-0.650661,-1.957443,2.634556,0.497564,5.279331,0.961598,0.533312,1.405250
7,0.270476,-0.164836,0.066836,-0.203763,5.262794,2.573425,-0.100214,2.505993,-0.576606,2.745836,0.319331,0.115563,0.209759,-0.211231,0.319331,1.422413,-0.263715,1.380251,-0.220501,1.422413,0.203255,-0.064622,-2.439157,0.372843,2.516958,0.492344,5.255838,0.953827,0.545160,0.319759
8,0.344783,0.121245,0.312484,0.080799,5.293169,2.513108,2.026127,-1.172836,0.913772,2.689452,0.325599,0.316295,-0.020734,-0.074447,0.325599,1.219322,0.838608,-0.633314,0.618382,1.219322,0.157676,-1.904882,1.485320,-0.832973,2.603716,0.500738,5.281927,0.961539,0.573878,6.039698
9,0.414408,0.239131,-0.219468,0.257652,5.292947,2.544326,-0.054367,1.632689,1.950632,2.718580,0.767043,0.181012,0.545009,0.508483,0.767043,0.426147,-0.260469,0.173934,0.288970,0.426147,0.492778,0.293498,-1.852158,-1.692980,2.574367,0.494526,5.276699,0.950948,0.514371,3.734324


###  We need to understand whether to put cuts on the dataset i.e. simply setting some if on the variables and if they are not respected eliminate the event.

## Step 2
I tried a feature scaling on the dataset.
Centered the points around their mean and scaled by their sample standard deviation. 
It seems that in general the algorithms of machine learning algorithms perform better after this kind of scaling.

In [5]:
from random import seed
from random import randint
# seed random number generator
n_seed=2347
seed(n_seed)

In [6]:
import sklearn
from sklearn.model_selection import train_test_split
Signal_comp['Type'] = 2 #Signal label == 2
Peaking_bkg_comp['Type'] = 1 # Peaking background label == 1
Continuum_bkg_comp['Type'] = 0 #Continuum backgroun label == 0
# Put in a dataframe randomly shuffled the lines
Sum_BS = pd.concat([Signal_comp,Peaking_bkg_comp,Continuum_bkg_comp]).sample(frac=1)
X = Sum_BS.drop('Type',axis=1)
Y = Sum_BS['Type']
# Dataframe divided into 80% of train data and 20% of test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8,random_state=randint(10**6,10**9))
# Standardize features by removing the mean and scaling to unit variance

Standardization of a dataset is a common requirement for many machine learning estimators: 
they might behave badly if the individual features do not more or less look like standard normally 
distributed data (e.g. Gaussian with 0 mean and unit variance). [More infos](https://scikit-learn.org/stable/modules/preprocessing.html)

In [7]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
print(ss.fit(X_train))
# The averages and standard deviations of all the databases are saved, 
# now with transfer I modify the data (Train and Test) in practice
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

# '''DNN Error: Input contains NaN, infinity or a value too large for dtype('float64')'''
# #Check if there are NaN in X_train and X_test
# Null_X_train=np.isnan(X_train)#.T.any().T.sum()
# print(Null_X_train)
# Null_X_test=np.isnan(X_test)#.T.any().T.sum()
# print(Null_X_test)


StandardScaler(copy=True, with_mean=True, with_std=True)


# Model Optimization

# ************** DRAFT **************
in "parameters" I would like to put also different possibilities for learning rate and solver at least. Maybe just 3 different learning rates and 2 different solvers, otherwise it could take too long to run

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
parameters = {'hidden_layer_sizes': [(100,),(100,100,),(100,100,100,), (200,300,100,), (100,200,300,100,)]}

mlp = MLPClassifier(max_iter=300, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=n_seed,
                    learning_rate_init=.1)

clf= GridSearchCV(mlp,param_grid=parameters,verbose=2,cv=5,return_train_score=True)

clf.fit(X_train, Y_train)

print ('RESULTS FOR NN\n')
print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
scores=pd.DataFrame(clf.cv_results_)
print(scores[['param_hidden_layer_sizes','mean_test_score']])

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] hidden_layer_sizes=(100,) .......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........................ hidden_layer_sizes=(100,), total=  16.6s
[CV] hidden_layer_sizes=(100,) .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.6s remaining:    0.0s


[CV] ........................ hidden_layer_sizes=(100,), total=  13.9s
[CV] hidden_layer_sizes=(100,) .......................................
[CV] ........................ hidden_layer_sizes=(100,), total=  15.9s
[CV] hidden_layer_sizes=(100,) .......................................
[CV] ........................ hidden_layer_sizes=(100,), total=  19.5s
[CV] hidden_layer_sizes=(100,) .......................................
[CV] ........................ hidden_layer_sizes=(100,), total=  22.5s
[CV] hidden_layer_sizes=(100, 100) ...................................
[CV] .................... hidden_layer_sizes=(100, 100), total=  14.0s
[CV] hidden_layer_sizes=(100, 100) ...................................
[CV] .................... hidden_layer_sizes=(100, 100), total=  15.5s
[CV] hidden_layer_sizes=(100, 100) ...................................
[CV] .................... hidden_layer_sizes=(100, 100), total=  16.2s
[CV] hidden_layer_sizes=(100, 100) ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 11.1min finished


RESULTS FOR NN

Best parameters set found:
{'hidden_layer_sizes': (200, 300, 100)}
Score with best parameters:
0.9844754934575294

All scores on the grid:
  param_hidden_layer_sizes  mean_test_score
0                   (100,)         0.982757
1               (100, 100)         0.983644
2          (100, 100, 100)         0.983145
3          (200, 300, 100)         0.984475
4     (100, 200, 300, 100)         0.984032


In [10]:
#get training and test error for the best NN model from CV

best_mlp = MLPClassifier(hidden_layer_sizes=(100, 200, 300, 100), max_iter=300, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=n_seed,
                    learning_rate_init=.1)

best_mlp.fit(X_train, Y_train)

training_error = 1. - best_mlp.score(X_train,Y_train)
test_error = 1. - best_mlp.score(X_test,Y_test)

print ('\nRESULTS FOR BEST NN\n')

print ("Best NN training error: %f" % training_error)
print ("Best NN test error: %f" % test_error)


RESULTS FOR BEST NN

Best NN training error: 0.000000
Best NN test error: 0.016630
